Importations


In [1]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.7/115.7 kB 13.1 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=af32ab68ba1176d29624fb2f2bdcec83ebe5585af3f98ed7293fce5571bde672
  Stored in directory: /root/.cache/pip/wheels/8b/67/f4/2caaae2146198dcb824f31a303833b07b14a5ec863fb3acd7b
Successfully built autograd-gamma


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from lifelines.utils import concordance_index

# ------------------ Load and Prep ------------------ #
df = pd.read_csv('/content/synthetic_nigeria_grid_data_2010_may2025_weekly_derived.csv', parse_dates=['Timestamp'])
df = df.sort_values('Timestamp').reset_index(drop=True)

# Feature engineering
df['Month'] = df['Timestamp'].dt.month
df['Year'] = df['Timestamp'].dt.year
df['Voltage_Lag1'] = df['Voltage (V)'].shift(1)
df['Current_Lag1'] = df['Current (A)'].shift(1)
df = df.bfill()

features = ['Voltage (V)', 'Current (A)', 'Transformer Fault', 'Line Trip Events',
            'Overload Condition', 'Year', 'Voltage_Lag1', 'Current_Lag1']
numerical = ['Voltage (V)', 'Current (A)', 'Voltage_Lag1', 'Current_Lag1']

# Scale numerical features
scaler = StandardScaler()
df[numerical] = scaler.fit_transform(df[numerical])

# Survival targets
df['Time'] = np.arange(len(df)) + 1
df['Event'] = df['Grid Collapse Events']
X = df[features]
y_time = df['Time'].astype(int).values
y_event = df['Event'].astype(int).values

X_train, X_test, time_train, time_test, event_train, event_test = train_test_split(
    X, y_time, y_event, test_size=0.2, random_state=42)

# ------------------ 1. lifelines CoxPH ------------------ #
cox_life = pd.concat([X_train.reset_index(drop=True),
                      pd.Series(time_train, name='Time'),
                      pd.Series(event_train, name='Event')], axis=1)

coxph_life = CoxPHFitter()
coxph_life.fit(cox_life, duration_col='Time', event_col='Event')
y_pred_lifelines = coxph_life.predict_partial_hazard(X_test)
cindex_lifelines = concordance_index(time_test, -y_pred_lifelines, event_test)

# ------------------ 2. XGBoost Survival ------------------ #
xgb_model = XGBRegressor(objective='survival:cox', n_estimators=100, learning_rate=0.05)
xgb_model.fit(X_train, time_train, sample_weight=event_train)
y_pred_xgb = xgb_model.predict(X_test)
cindex_xgb = concordance_index(time_test, -y_pred_xgb, event_test)

# ------------------ Compare Models ------------------ #
print("\nConcordance Index Scores")
print(f"1. lifelines CoxPH:    {cindex_lifelines:.4f}")
print(f"2. XGBoost Survival:   {cindex_xgb:.4f}")

# ------------------ Optional: Bar Plot ------------------ #
import seaborn as sns

results = pd.DataFrame({
    'Model': ['Lifelines CoxPH', 'XGBoost Survival'],
    'Concordance Index': [cindex_lifelines, cindex_xgb]
}).sort_values('Concordance Index', ascending=False)

plt.figure(figsize=(8, 4))
sns.barplot(data=results, x='Model', y='Concordance Index', palette='viridis')
plt.ylim(0.5, 1.0)
plt.title('Survival Model Comparison')
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/synthetic_nigeria_grid_data_2010_may2025_weekly_derived.csv'

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# ------------------ Load & Preprocess ------------------ #
df = pd.read_csv('/content/synthetic_nigeria_grid_data_2010_may2025_daily_derived.csv', parse_dates=['Timestamp'])
df = df.sort_values('Timestamp').reset_index(drop=True)

# Feature engineering
df['Month'] = df['Timestamp'].dt.month
df['Year'] = df['Timestamp'].dt.year
df['Voltage_Lag1'] = df['Voltage (V)'].shift(1)
df['Current_Lag1'] = df['Current (A)'].shift(1)
df = df.bfill()

# Define features
features = ['Voltage (V)', 'Current (A)', 'Transformer Fault', 'Line Trip Events',
            'Overload Condition', 'Year', 'Voltage_Lag1', 'Current_Lag1']
numerical = ['Voltage (V)', 'Current (A)', 'Voltage_Lag1', 'Current_Lag1']

# Scale numeric features
scaler = StandardScaler()
df[numerical] = scaler.fit_transform(df[numerical])

# Survival targets
df['Time'] = np.arange(len(df)) + 1
df['Event'] = df['Grid Collapse Events']

X = df[features]
y_time = df['Time'].astype(int).values
y_event = df['Event'].astype(int).values

# Train/Test Split
X_train, X_test, time_train, time_test, event_train, event_test, df_train, df_test = train_test_split(
    X, y_time, y_event, df, test_size=0.2, random_state=42)

# ------------------ Train CoxPH Model ------------------ #
cox_input = pd.concat([X_train.reset_index(drop=True),
                       pd.Series(time_train, name='Time'),
                       pd.Series(event_train, name='Event')], axis=1)

coxph = CoxPHFitter(penalizer=0.1)
coxph.fit(cox_input, duration_col='Time', event_col='Event')

cindex = concordance_index(time_test, -coxph.predict_partial_hazard(X_test), event_test)
print(f"\nCoxPH Concordance Index: {cindex:.4f}")

# ------------------ Predict Hazard & Survival ------------------ #
df_test = df_test.reset_index(drop=True)
surv_funcs = coxph.predict_survival_function(X_test)
df_test['Hazard_Score'] = coxph.predict_partial_hazard(X_test).values
df_test['Survival_At_Week_10'] = surv_funcs.loc[10].values

# ------------------ Top-10 High-Risk Weeks ------------------ #
top_k = 10
high_risk_weeks = df_test.sort_values('Hazard_Score', ascending=False).head(top_k)

print(f"\nTop-{top_k} Weeks with Highest Risk of Grid Collapse:")
print(high_risk_weeks[['Timestamp', 'Hazard_Score', 'Survival_At_Week_10']])

# ------------------ Feature Diagnosis per Risky Week ------------------ #
# ------------------ Feature Diagnosis per Risky Week ------------------ #
def generate_diagnoses_df(top_df, reference_df, feature_list, threshold_std=1.0):
    stats = reference_df[feature_list].agg(['mean', 'std']).T
    diagnosis_list = []

    for idx, row in top_df.iterrows():
        timestamp = row['Timestamp']
        week_summary = {'Timestamp': timestamp, 'Hazard_Score': row['Hazard_Score'],
                        'Survival_At_Week_10': row['Survival_At_Week_10'], 'Abnormal_Features': []}

        for feat in feature_list:
            val = row[feat]
            mean = stats.loc[feat, 'mean']
            std = stats.loc[feat, 'std']
            z = (val - mean) / std

            if z > threshold_std:
                week_summary['Abnormal_Features'].append(f"High {feat} (z={z:.2f})")
            elif z < -threshold_std:
                week_summary['Abnormal_Features'].append(f"Low {feat} (z={z:.2f})")

        week_summary['Abnormal_Features'] = ', '.join(week_summary['Abnormal_Features']) if week_summary['Abnormal_Features'] else 'None'
        diagnosis_list.append(week_summary)

    return pd.DataFrame(diagnosis_list)

diagnosis_df = generate_diagnoses_df(high_risk_weeks, df, features)

# Show result
print("\nDiagnosis DataFrame for Top Hazard Weeks:")
print(diagnosis_df[['Timestamp', 'Hazard_Score', 'Survival_At_Week_10', 'Abnormal_Features']])

In [3]:
plt.figure(figsize=(12, 5))
plt.plot(df['Timestamp'], coxph.predict_partial_hazard(df[features]))
plt.title("Hazard Score Over Time (All Weeks)")
plt.xlabel("Timestamp")
plt.ylabel("Hazard Score")
plt.grid(True)
plt.tight_layout()
plt.show()


NameError: name 'df' is not defined

<Figure size 1200x500 with 0 Axes>

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_curve
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LSTM, Conv1D, Flatten, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import backend as K

# ------------------ 1. Load and Engineer ------------------ #
df = pd.read_csv("/content/synthetic_nigeria_grid_data_2010_may2025_weekly_derived.csv", parse_dates=['Timestamp'])

df['Year']      = df['Timestamp'].dt.year
df['Month']     = df['Timestamp'].dt.month
df['Weekday']   = df['Timestamp'].dt.weekday
df['IsWeekend'] = df['Weekday'].isin([5, 6]).astype(int)

df['Next_Week_Collapse'] = df['Grid Collapse Events'].shift(-1)
df.dropna(inplace=True)
df.sort_values("Timestamp", inplace=True)

# ------------------ 2. Train/Test Split ------------------ #
target = 'Next_Week_Collapse'
X = df.drop(columns=['Timestamp', 'Grid Collapse Events', target])
y = df[target]

train_mask = df['Year'] < 2023
X_train, y_train = X[train_mask], y[train_mask]
X_test,  y_test  = X[~train_mask], y[~train_mask]

# ------------------ 3. Preprocessing ------------------ #
categorical = ['Transformer Fault', 'Line Trip Events', 'Overload Condition', 'IsWeekend', 'Weekday', 'Month']
numerical = list(set(X.columns) - set(categorical))

preprocessor = ColumnTransformer([
    ('num', MinMaxScaler(), numerical),
    ('cat', OneHotEncoder(drop='first'), categorical)
])

X_train_p = preprocessor.fit_transform(X_train)
X_test_p = preprocessor.transform(X_test)

# ------------------ 4. Sequence Creation ------------------ #
def make_sequences(X, y, window=6):
    X_seq, y_seq = [], []
    for i in range(len(X) - window):
        X_seq.append(X[i:i+window])
        y_seq.append(y[i+window])
    return np.array(X_seq), np.array(y_seq)

window_size = 24
X_train_seq, y_train_seq = make_sequences(X_train_p, y_train.values, window_size)
X_test_seq, y_test_seq = make_sequences(X_test_p, y_test.values, window_size)

# ------------------ 5. Weighted BCE Loss ------------------ #
def weighted_bce(pos_weight):
    def loss(y_true, y_pred):
        bce = K.binary_crossentropy(y_true, y_pred)
        return K.mean(bce * (y_true * pos_weight + (1 - y_true)))
    return loss

pos_weight = .0
loss_fn = weighted_bce(pos_weight)

# ------------------ 6. Build Models ------------------ #
def build_tcn(shape):
    inp = Input(shape=shape)
    x = Conv1D(64, 3, padding='causal', activation='relu')(inp)
    x = Dropout(0.2)(x)
    x = Conv1D(32, 3, padding='causal', activation='relu')(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    return Model(inp, out)

def build_lstm(shape):
    inp = Input(shape=shape)
    x = LSTM(64, return_sequences=True)(inp)
    x = Dropout(0.2)(x)
    x = LSTM(32)(x)
    x = Dense(64, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    return Model(inp, out)

shape = X_train_seq.shape[1:]
tcn_model = build_tcn(shape)
lstm_model = build_lstm(shape)

tcn_model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
lstm_model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

# ------------------ 7. Train ------------------ #
callbacks = [EarlyStopping(patience=5, restore_best_weights=True)]

tcn_model.fit(X_train_seq, y_train_seq, epochs=20, batch_size=32,
              validation_split=0.2, callbacks=callbacks, verbose=1)

lstm_model.fit(X_train_seq, y_train_seq, epochs=20, batch_size=32,
               validation_split=0.2, callbacks=callbacks, verbose=1)

# ------------------ 8. Rolling Prediction ------------------ #
probs, true_labels = [], []

for i in range(len(X_test_seq)):
    x = X_test_seq[i:i+1]
    p1 = tcn_model.predict(x, verbose=0)[0][0]
    p2 = lstm_model.predict(x, verbose=0)[0][0]
    probs.append((p1 + p2) / 2)
    true_labels.append(y_test_seq[i])

probs = np.array(probs)
y_true = np.array(true_labels).astype(int)

# ------------------ 9. Threshold Sweep Plot ------------------ #
precisions, recalls, thresholds = precision_recall_curve(y_true, probs)

'''plt.figure(figsize=(8, 5))
plt.plot(thresholds, precisions[:-1], label='Precision', linewidth=2)
plt.plot(thresholds, recalls[:-1], label='Recall', linewidth=2)
plt.axvline(x=0.45, color='gray', linestyle='--', label='Current Threshold (0.45)')
plt.title("Precision-Recall vs Threshold")
plt.xlabel("Threshold")
plt.ylabel("Score")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()'''

# ------------------ 10. Evaluation ------------------ #
threshold = 0.5
y_pred = (probs >= threshold).astype(int)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4))

cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['No Collapse', 'Collapse'],
            yticklabels=['No Collapse', 'Collapse'])
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/synthetic_nigeria_grid_data_2010_may2025_weekly_derived.csv'

EDA